In [ ]:
'''Notebook for generation of cluster algebra exchange graphs, performing their network analysis, and generating data for ML'''
#Import libraries
import numpy as np
from math import comb
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx 


In [ ]:
#Define Exchange Graph function for cluster mutation
def ExchangeGraph(init_seed,depth,print_check=True):  
    '''Input:  inital seed as a ClusterSeed(...) or ClusterSeed(...).quiver(), and the max depth to mutate to, can choose whether to output progress checks.
       Output: exchange graph generated to the specified depth as a networkx graph, a list of all seeds (with index matching the graph vertex labels).'''
    #Output initial seed and save number of cluster variables (i.e. nodes)
    if print_check: print('Initial_seed: '+str(init_seed)+'\n')
    n = init_seed.n()                #...extract number of mutable variables
    
    #Initialise EG and lists to keep track of all seeds, those requiring mutation, and indices of those to check against in previous 2 depths
    EG = nx.empty_graph()            #...define the exchange graph
    EG.add_node(0)                   #...add node corresponding to initial seed
    seed_list, current_seeds, next_seeds, checkable_seeds = [init_seed], [], [[init_seed,0,-1]], [0,0]
    if print_check: print('Progress report:\n{:<14}  {:<20}  {:<20}'.format('Depth: 0,', '# new seeds: -', '# total seeds: '+str(len(seed_list)))) #...output mutation updates
    
    #Mutate initial seed to specified depth
    for d in range(1,depth+1):
        #Update the current seeds to mutate, and the seeds in previous 2 depths to check against
        current_seeds = next_seeds
        next_seeds = []
        checkable_seeds[0] = checkable_seeds[1]                #...update list position of start of 2 depths ago (only need to check these for equivalence)
        checkable_seeds[1] = len(seed_list)-len(current_seeds) #...save end of last depth position for next iteration
        #Loop through the current seeds, mutating all their other variables
        for seed_info in current_seeds:
            #Identify the nodes to mutate about (if first depth then mutate them all)
            if d == 1: variable_list = list(range(n))
            else: variable_list = list(range(n))[:seed_info[2]]+list(range(n))[seed_info[2]+1:] #...skip last vertex mutated about
            #Mutate about all nodes not previously mutated about
            for variable in variable_list: 
                new_seed = seed_info[0].mutate(variable,inplace=False)
                new_test = True              #...boolean to verify if the generated seed is new
                #Loop through all previous depth and new seeds in EG, check if the new seed matches any of them, if so add edge from current seed, otherwise add as a new node & edge
                for idx, old_seed in enumerate(seed_list[checkable_seeds[0]:]): #...idx gives the seeds label in EG, note includes checking where 2 mutations of a seed produce the same seed
                    if old_seed == new_seed: #...only checks NOT up to equivalence (i.e no shuffling cluster variables)
                        new_test = False     #...note that the generated seed is a repeat
                        #If edge already in EG, add extra mutation label if relevant (i.e. not already in label's list)
                        if EG.has_edge(min(seed_info[1],idx+checkable_seeds[0]),max(seed_info[1],idx+checkable_seeds[0])):
                            if variable not in list(nx.get_edge_attributes(EG,'label')[min(seed_info[1],idx+checkable_seeds[0]),max(seed_info[1],idx+checkable_seeds[0])]):
                                #Where edge already in EG and label needs updatng to include new mutable node, remove old edge (required otherwise nx overlaps), then readd edge with updated label - occurs for quiver EGs only
                                new_label = [*list(nx.get_edge_attributes(EG,'label')[min(seed_info[1],idx+checkable_seeds[0]),max(seed_info[1],idx+checkable_seeds[0])]),variable]
                                EG.remove_edge(min(seed_info[1],idx+checkable_seeds[0]),max(seed_info[1],idx+checkable_seeds[0]))
                                EG.add_edge(min(seed_info[1],idx+checkable_seeds[0]),max(seed_info[1],idx+checkable_seeds[0]),label=new_label)
                        #Add new edge involving old nodes
                        else: 
                            EG.add_edge(min(seed_info[1],idx+checkable_seeds[0]),max(seed_info[1],idx+checkable_seeds[0]),label=[variable]) #...add edge connecting to relevant seed
                        break   
                if new_test:
                    seed_list.append(new_seed)
                    next_seeds.append([new_seed,len(seed_list)-1,variable])      #...save new seed to mutate, the seed's label in the EG, and the cluster variable to not mutate in next iteration
                    EG.add_edge(seed_info[1],len(seed_list)-1,label=[variable])  #...add new node, connected to mutated seed
        if print_check: print('{:<14}  {:<20}  {:<20}'.format('Depth: '+str(d)+',', '# new seeds: '+str(len(next_seeds))+',', '# total seeds: '+str(len(seed_list)))) #...output mutation updates
        #Check if any new seeds were generated at this depth, otherwise terminate loop
        if len(next_seeds) == 0:
            if print_check:
                try: init_seed.cluster(); print('\n...early termination, no new seeds found at depth:',d,'--> finite type') #...if input is a cluster then must be finite type
                except: print('\n...early termination, no new seeds found at depth:',d,'--> finite-mutation type') #...if input is a quiver then may just be finite-mutation type
            break
            
    return [EG,seed_list] #...note the EG vertex labelling matches indices in the returned seed_list


########### Exchange Graph Analysis ###########

In [ ]:
#Exchange Graph Analysis
#EGs examined take form: ClusterSeed(['A',4]), ClusterSeed(['B',4]), ClusterSeed(['C',4]), ClusterSeed(['D',4]), ClusterSeed(['F',4]), ClusterSeed(['A',[1,3],1]), ClusterSeed(['A',[2,2],1]), ClusterSeed(matrix([[0,2,0,0],[-2,0,1,0],[0,-1,0,1],[0,0,-1,0]])), ClusterSeed(matrix([[0,2,0,-2],[-2,0,2,0],[0,-2,0,1],[2,0,-1,0]])) 
G = ClusterSeed(['A',4]) #... add '.quiver()' after to instead generate the respective quiver EG
G_EG = ExchangeGraph(G, 4, True)

#Run full network analysis and output results
print('Density:',nx.density(G_EG[0]),'for',len(G_EG[0]),'nodes')
print('Clustering (triangle,square):','('+str(sum(nx.clustering(G_EG[0]).values())/float(len(G_EG[0])))+','+str(sum(nx.square_clustering(G_EG[0]).values())/float(len(G_EG[0])))+')')
WIndex = nx.wiener_index(G_EG[0])
print('Wiener index:',WIndex,'(normalised:',WIndex/comb(len(G_EG[0]),2),')')
EVCentrality = nx.eigenvector_centrality(G_EG[0],max_iter=1000)
max_EVC = max(EVCentrality, key=EVCentrality.get)
print('Centrality:',max_EVC)
print('...centrality range:',max(EVCentrality.values())-min(EVCentrality.values()))
if max_EVC == 0: print('...depth-step centrality change:',EVCentrality[0]-max([EVCentrality[i] for i in range(1,5)]))
cycle_basis = nx.minimum_cycle_basis(G_EG[0])
cycle_lengths, freqs = np.unique(np.array(list(map(len,cycle_basis))), return_counts=True)
print('Cycle basis info (cycle length, frequency):\n',np.asarray((cycle_lengths, freqs)).T)


In [ ]:
#Output EG with mutation labels
G = ClusterSeed(['A',4])
G_EG = ExchangeGraph(G,4,False)
    
#Draw the EG with vertices & edges labelled
if len(G_EG[1]) < 100:    #...draw EG if suitably small
    pos=nx.kamada_kawai_layout(G_EG[0])
    plt.figure('EG',figsize=(6,6)) 
    nx.draw(G_EG[0],pos,with_labels=True) #...set to with_labels=False to omit vertex labels
    nx.draw_networkx_edge_labels(G_EG[0],pos,edge_labels=nx.get_edge_attributes(G_EG[0],'label'),label_pos=0.5)     #...comment out this line to omit the edge labels
    #plt.savefig('./__.pdf')
    print('\nNumber of seeds:',len(G_EG[1]))
print('EG degree frequency distribution:',nx.degree_histogram(G_EG[0]))


In [ ]:
#Plot Cluster EG with nodes coloured st all those with the same colour have the same quiver
#Define variables to run with
cycle_subgraph = False        #...choose whether to output full EG or just subgraph corresponding to selected 'quivers_to_colour' (usually a cycle)
use_quiver_EG = True          #...choose whether to also output the quiver EG
all_coloured_check = False    #...choose whether to colour all the different quiver differently (True), or just quivers of choice (False)
quivers_to_colour = [0,3,8,2] #...if just colour selected quivers (all_coloured_check=False), choose which in quiver list will be coloured
preset_colours = ['red','blue','orange','green','yellow','purple','gray','brown'] #...some example colours, use all_coloured_check=True if wish to colour all (i.e. more than 8)

##################################
#Define initial seed and its EG
depth = 4     #...can set a high depth (say >100) so generates entire EG for finite-type case
G = ClusterSeed(['A',4]) 
G_EG = ExchangeGraph(G, depth, True)  #...compute exchange graph

#If wish to use Quiver EG (use_quiver_EG=True), compute it and save its quivers to compare to
if use_quiver_EG: 
    Quiv_EG = ExchangeGraph(G.quiver(), depth, False)
    quivers = Quiv_EG[1]
else: quivers = []

#Allocate seeds an integer based on the quiver they're related to (integer will then relate to the colour in the colour_map)
seed_colours = []
for seed in G_EG[1]:
    if seed.quiver() in quivers:
        seed_colours.append(quivers.index(seed.quiver()))
    else:
        quivers.append(seed.quiver())
        if len(seed_colours) == 0: seed_colours.append(0)
        else: seed_colours.append(max(seed_colours)+1)

#Allocate colours to all quivers in the EG
if all_coloured_check: 
    #Normalise the colour map and then plot
    low, *_, high = sorted(seed_colours)
    norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
    mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
    colour_map = [mapper.to_rgba(i) for i in seed_colours]
    #...note the darker colours are lower integers in list, which are then quivers found earlier in the checking, st they are roughly quivers closer to the EG centre
#Only allocate colours to selected quivers in the EG (all others black)
else: 
    colour_map = []
    coloured_clusts = [[] for i in quivers_to_colour]
    for seed_idx, colour_idx in enumerate(seed_colours):
        if colour_idx in quivers_to_colour: 
            colour_map.append(preset_colours[quivers_to_colour.index(colour_idx)])
            coloured_clusts[quivers_to_colour.index(colour_idx)].append([seed_idx,G_EG[1][seed_idx].cluster()])
        else: colour_map.append('black')

#Output the figures
print('Initial_seed: '+str(G_EG[1][0])+'\nB-matrix:\n'+str(G_EG[1][0].b_matrix()))
#If wish to output the Quiver EG for comparison
if use_quiver_EG:
    if not all_coloured_check: print('\nQuiver EG seeds highlighted:',quivers_to_colour,'-->',preset_colours[:len(quivers_to_colour)])
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(40, 20))
    Qpos = nx.kamada_kawai_layout(Quiv_EG[0])
    nx.draw(Quiv_EG[0], Qpos, with_labels=True, ax=axes[0]) #...output Quiver EG for comparison
    nx.draw_networkx_edge_labels(Quiv_EG[0],Qpos,edge_labels=nx.get_edge_attributes(Quiv_EG[0],'label'),label_pos=0.5, ax=axes[0])
    if cycle_subgraph: 
        cycle_sub = G_EG[0].subgraph([clust[0] for colour in coloured_clusts for clust in colour])
        pos=nx.kamada_kawai_layout(cycle_sub) #...can change to nx.spring_layout(cycle_sub) for example
        nx.draw(cycle_sub, pos, with_labels=True, ax=axes[1])
        nx.draw_networkx_edge_labels(cycle_sub,pos,edge_labels=nx.get_edge_attributes(cycle_sub,'label'),label_pos=0.4, ax=axes[1])     #...add edge features
    else: nx.draw_kamada_kawai(G_EG[0], with_labels=True, node_color=colour_map, font_color='white', ax=axes[1])
    axes[0].title.set_text('Quiver EG')
    axes[1].title.set_text('Cluster EG')
    fig.tight_layout() 
#Otherwise just output the seed EG (or just the selected quivers respective subgraph)
else: 
    if cycle_subgraph:
        cycle_sub = G_EG[0].subgraph([clust[0] for colour in coloured_clusts for clust in colour])
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
        pos=nx.kamada_kawai_layout(cycle_sub) #...can change to nx.spring_layout(cycle_sub) for example
        nx.draw(cycle_sub,pos, with_labels=True)
        nx.draw_networkx_edge_labels(cycle_sub,pos,edge_labels=nx.get_edge_attributes(cycle_sub,'label'),label_pos=0.5)     #...add edge features
        subgraph_cb = nx.minimum_cycle_basis(cycle_sub) #...minimum cycle basis of cycle-subgraph will just be the cycles
        print('\nSubgraph cycles: (#,lengths) --> ',len(subgraph_cb),list(map(len,subgraph_cb)),'\n',subgraph_cb,'\n') 
    else:   
        nx.draw_kamada_kawai(G_EG[0], with_labels=True, node_color=colour_map, font_color='white')
    plt.tight_layout()
    #plt.savefig('./A4cycle'+str(quivers_to_colour)+'.pdf')

''' #...uncomment to also output the respective quivers & clusters in the cycle
#Output the quivers and clusters in the cycle
print(np.array([quivers[q_idx].b_matrix() for q_idx in quivers_to_colour]),end='\n\n')
for q_clustslist in coloured_clusts: 
    for clust in q_clustslist:
        print(clust) 
    print('') 
'''


In [ ]:
#Analyse how cycle basis of QEG maps in EG
G = ClusterSeed(['A',4]) 
EG = ExchangeGraph(G, 100, False) #...set depth high so generates entire finite type EG
QEG = ExchangeGraph(G.quiver(), 100, False)
qeg_mcb = nx.minimum_cycle_basis(QEG[0])
print('QEG basis length:',len(qeg_mcb),'\nqeg_mcb ((lengths,counts)):',[list(i) for i in np.unique([len(cyc) for cyc in qeg_mcb],return_counts=True)])

#See how QEG minimum cycle basis embeds in EG
p,q,subs=[],[],[] #...lists to record the scaling (p) and copying (q) factors for the quiver cycle embedding, aswell as the full embedded subgraph (subs)
for cyc in qeg_mcb:
    quivs = [QEG[1][v].b_matrix() for v in cyc]
    nodes = []
    for seed in range(len(EG[1])):
        if EG[1][seed].b_matrix() in quivs: nodes.append(seed)
    eg_subgraph = EG[0].subgraph(nodes)
    eg_sub_mcb = nx.minimum_cycle_basis(eg_subgraph)
    p.append(int(len(eg_sub_mcb[0])/len(cyc)))
    q.append(len(eg_sub_mcb))
    subs.append(eg_subgraph)

print('p:',[list(i) for i in np.unique(p,return_counts=True)],'\nq:',[list(i) for i in np.unique(q,return_counts=True)])


In [ ]:
#Plot EG cycle basis length vs depth
relative_to_number_of_seeds = False #...plot the basis lengths relative to the number of seeds in the EG
quiver_include = False              #...include the repsecitve QEGs in the analysis
#Select initial seeds to generate algebras for (note omit the infinite type if wish to go to larger depths)
init_seeds = [ClusterSeed(['A',4]),ClusterSeed(['D',4]),ClusterSeed(['F',4]),ClusterSeed(['A',[1,3],1]),ClusterSeed(['A',[2,2],1]),ClusterSeed(matrix([[0,2,0,0],[-2,0,1,0],[0,-1,0,1],[0,0,-1,0]])),ClusterSeed(matrix([[0,2,0,-2],[-2,0,2,0],[0,-2,0,1],[2,0,-1,0]]))]
type_labels = ['A4','D4','F4','A13','A22','I1','I2']
style = ['-','-','-','-.','-.','--','--']
depth = 4

#Save cycle basis lengths for selected EGs to specified depth
cycle_basis_lengths = []
for seed in init_seeds:
    if quiver_include: cycle_basis_lengths.append([[0],[0]]) #...at depth 0 there're no cycles
    else:              cycle_basis_lengths.append([[0]])
    for d in range(1,depth+1):
        if quiver_include: EG_quiv = ExchangeGraph(seed.quiver(), d, False)
        EG = ExchangeGraph(seed, d, False)
        if relative_to_number_of_seeds:
            cycle_basis_lengths[-1][0].append(len(nx.minimum_cycle_basis(EG[0]))/len(EG[1]))
            if quiver_include: cycle_basis_lengths[-1][1].append(len(nx.minimum_cycle_basis(EG_quiv[0]))/len(EG_quiv[1]))
        else:
            cycle_basis_lengths[-1][0].append(len(nx.minimum_cycle_basis(EG[0])))
            #cycle_basis_lengths[-1][0].append(max([len(x) for x in nx.minimum_cycle_basis(EG[0])]+[0]))  #...can use instead of above line to consider maximum cycle size with depth --> trivially 4 except A13
            if quiver_include: cycle_basis_lengths[-1][1].append(len(nx.minimum_cycle_basis(EG_quiv[0]))) #max([len(x) for x in nx.minimum_cycle_basis(EG_quiv[0])]+[0]) #...as above the line substitute to make if considering maximum cycle size

#Plot the graph
plt.figure('Cycle Basis Lengths')
for seed_idx in range(len(cycle_basis_lengths)):
    plt.plot(range(depth+1),cycle_basis_lengths[seed_idx][0],label=type_labels[seed_idx],linestyle=style[seed_idx])
    if quiver_include: plt.plot(range(depth+1),cycle_basis_lengths[seed_idx][1],label=type_labels[seed_idx]+' (quiver)')
plt.xlabel('Depth')
plt.xticks(range(depth+1))
if relative_to_number_of_seeds: 
    plt.ylabel('Cycle Basis Length / Number of Seeds')
    plt.yticks(np.linspace(0,1,11))
else: 
    plt.ylabel('Cycle Basis Length') #plt.ylabel('Max Cycle Basis Length')
    plt.yticks(range(0,max(np.ndarray.flatten(np.array(cycle_basis_lengths)))+5,5)) #...change to a suitable step size
plt.legend(loc='upper left')
plt.grid()
#plt.savefig('__.pdf')


########### ML Data Generation ###########

In [ ]:
#Define functions to convert each algebra seeds to a tensor
#Function to convert a cluster variable into a tensor
def variable_to_tensor(var,num_variables,sparse_check=True):
    '''Inputs:  the cluster variable to convert (a Laurent polynomial), the algebra rank (i.e. cluster size / number of variables per seed), option to convert to sparse format (or full form when False)
       Outputs: the cluster variable as a tensor in the chosen form (with the maximum monomial degree in numerator and denominator for full form, to speed up padding)'''
    if '-' in str(var): print('Negative coefficient error')  #...does not occur for considered algebras
    
    init_variables = ['x'+str(i) for i in range(num_variables)]
    #Convert to a string, remove brackets and split into num/denom
    var_num_denom = str(var).replace('(','').replace(')','').replace(' ','').split('/') 
    #Split into monomial terms
    var_monomials = [var_num_denom[i].split('+') for i in range(len(var_num_denom))] #...if '-' occur, add .replace('-','+') before split? make a '-1' entry in 
    #Split monomials up into variables, and identify constants at start of list (or add a constant of 1 if not one there)
    for i in range(len(var_monomials)): 
        for m in range(len(var_monomials[i])): 
            var_monomials[i][m] = var_monomials[i][m].split('*')
            if not var_monomials[i][m][0].isdigit(): var_monomials[i][m].insert(0,1) #..add a coefficient at start of each monomial list if not already one
            else: var_monomials[i][m][0] = int(var_monomials[i][m][0]) #...if already a coefficient then convert it to an integer
    #Convert powers to products of as many variables
    if '^' in str(var):
        for nd in range(len(var_monomials)):
            for mono in range(len(var_monomials[nd])):
                new_mono = [var_monomials[nd][mono][0]]
                for term in range(1,len(var_monomials[nd][mono])):
                    if '^' in var_monomials[nd][mono][term]:
                        temp=var_monomials[nd][mono][term].split('^')
                        new_mono += [temp[0] for i in range(int(temp[1]))]
                var_monomials[nd][mono] = new_mono

    #Return a sparse 'coo' rep of tensor if requested (each list a monomial with first entry the coefficient then subsequent entries the power each variable is raised to in the monomial)
    if sparse_check:
        tensor = [[],[]]
        for monomial in var_monomials[0]: 
            tensor[0].append([monomial[0]]+[monomial.count(init_variables[i]) for i in range(num_variables)])
        #If no denom add a vector of zeros
        if len(var_monomials) == 1: tensor[1].append(list(np.zeros(num_variables+1,dtype=int)))
        else: 
            for monomial in var_monomials[1]: 
                tensor[1].append([monomial[0]]+[monomial.count(init_variables[i]) for i in range(num_variables)])
        return tensor
    #Return a (num-variables)-dimensional tensor (2 of them for num and denom independently), the number of dimensions is the number of cluster variables, the size of the dimension is the largest degree of the polynomials + 1 (for constants)
    else:
        #Extract the max degree of the monomials in num then denom
        monomial_max_degrees = [max([len(x)-1 for x in numdenom]) for numdenom in var_monomials]
        #Construct an empty tensor up to the degree of the num & denom
        #tensor = [np.zeros(np.sum([comb(num_variables+d-1,d) for d in range(max_degree+1)]),dtype=int) for max_degree in monomial_max_degrees]
        tensor = [np.zeros(tuple(max_degree+1 for i in range(num_variables)),dtype=int) for max_degree in monomial_max_degrees]
        if len(tensor) == 1: tensor.append(np.array([],dtype=int)) #...if there's no denominator, represent with an empty list
        #Add monomial terms into the tensor for both numerator and denominator
        for numdenom in range(len(var_monomials)):
            for monomial in var_monomials[numdenom]: 
                index = [monomial.count(init_variables[i]) for i in range(num_variables)]
                #print('monomial to add:',monomial,index)
                tensor[numdenom][tuple(index)] = monomial[0]
        return tensor, monomial_max_degrees

#Define function to compute the tensors of all variables in a cluster
def cluster_to_tensors(clust,sparse_check=True):
    '''Inputs:  the cluster to convert, an option to convert to sparse tensor form (preferred) or full form (note 'sparse form' means the form to adapt to sparse tensors which hears produces less sparse tensors)
       Outputs: the cluster as a tensor in the specified form (with maximum monomial degrees across all variables in cluster if full form, to speed up padding)'''
    #Construct the tensors for each cluster variable
    if sparse_check: 
        clust_tensors = []
        for var in clust:
            clust_tensors.append(variable_to_tensor(var,len(clust),sparse_check=sparse_check))
        return clust_tensors
    if not sparse_check: 
        clust_tensors, max_degrees = [], []
        for var in clust:
            output = variable_to_tensor(var,len(clust),sparse_check=sparse_check)
            clust_tensors.append(output[0])
            max_degrees.append(output[1])
        return clust_tensors, max_degrees


In [ ]:
#Generate cluster tensor data, and output to a txt file
save_data = False     #...choose whether to save the generated data
sparse_check = True   #...choose which data form to use: True --> 'coo' sparse adatped form (preferred!), False --> full tensor form

#Define clusters to generate data for 
seeds = [ClusterSeed(['A',4],ClusterSeed(['D',4]) #...select a set of algebras to generate data for (for real vs fake ML just select 1 algebra)
depths = [4,4]                                    #...specify the depths to generate each algebra too
#Generate the EGs
EGs = [ExchangeGraph(seeds[i], depths[i], False) for i in range(len(seeds))]
class_sizes = [len(EG[1]) for EG in EGs]
print('Class sizes:',class_sizes,'-->',sum(class_sizes))
clusters_list = []
for EG in EGs: clusters_list += EG[1]      #...concatenate all the seeds into one dataset
number_variables = len(seeds[0].cluster()) #...save the rank for the function input

#Convert the clusters to tensors
tclusters,tclusters_sizes = [],[0,0] 
for clust in clusters_list:
    output = cluster_to_tensors(clust.cluster(),sparse_check=sparse_check)
    if sparse_check:         #...in sparse case just keep clusters as tensors and pad at end
        tclusters.append([output,clust.b_matrix()])
        #Save the maximum length of num & denom
        for var in output:
            if len(var[0]) > tclusters_sizes[0]: tclusters_sizes[0] = len(var[0])
    else: 
        tclusters.append([output[0],clust.b_matrix()])
        #Update the maximum num and denom sizes if necessary (for padding)
        for var in output[1]:
            if var[0] > tclusters_sizes[0]: tclusters_sizes[0] = var[0]
            if len(var) > 1: #...if denomiator list exists
                if var[1] > tclusters_sizes[1]: tclusters_sizes[1] = var[1]

if sparse_check: print('Cluster numerator sizes:',tclusters_sizes[0])
else: print('Cluster numerator sizes:',tclusters_sizes[0]+1)

#Padding       
#Flatten then pad the sparse-form tensor numerators to max sizes (so consistent tensor sizes), then concatenate 
if sparse_check:
    for c_idx in range(len(tclusters)): #...loop through clusters
        for v_idx in range(number_variables): #...loop through cluster variables
            #Pad Numerators (denominators always one monomial so no need to pad)
            while len(tclusters[c_idx][0][v_idx][0]) < tclusters_sizes[0]: #...each clust in tclusters is [[variable[num[monomial],denom[monomial]]],exchange]
                tclusters[c_idx][0][v_idx][0].append(list(np.zeros(number_variables+1,dtype='int')))
        tclusters[c_idx] = np.concatenate((np.array(tclusters[c_idx][1]).flatten(),np.array([np.concatenate((np.array(tclusters[c_idx][0][v_idx][0]).flatten(),np.array(tclusters[c_idx][0][v_idx][1]).flatten()),axis=0) for v_idx in range(number_variables)]).flatten()))
    print('Tensor length:',len(tclusters[0])) #...lengths should all be exchange_matrix + (num_variables)*(num_variables+1)*(max_num_numerator_monomials+max_num_denominator_monomials)

#Pad all the full-form tensors to the maximum sizes (of num & denom)
else:
    for c_idx in range(len(tclusters)): #...loop through clusters
        for v_idx in range(len(tclusters[c_idx][0])): #...loop through variables in each cluster
            #Numerators
            if tclusters[c_idx][0][v_idx][0].shape[0] < tclusters_sizes[0]+1:
                tclusters[c_idx][0][v_idx][0] = np.pad(tclusters[c_idx][0][v_idx][0],((0,tclusters_sizes[0]+1-tclusters[c_idx][0][v_idx][0].shape[0]),),mode='constant')
            #Denominators 
            #If no denominator create an empty array with correct dimensions
            if len(tclusters[c_idx][0][v_idx][1]) == 0: tclusters[c_idx][0][v_idx][1] = np.zeros(tuple(tclusters_sizes[1]+1 for i in range(number_variables)))
            elif tclusters[c_idx][0][v_idx][1].shape[0] < tclusters_sizes[1]+1:
                tclusters[c_idx][0][v_idx][1] = np.pad(tclusters[c_idx][0][v_idx][1],((0,tclusters_sizes[1]+1-tclusters[c_idx][0][v_idx][1].shape[0]),),mode='constant')
    #Flatten and concatenate all the variable tensors, with exchange matrices, for each cluster
    for c_idx in range(len(tclusters)): #...loop through clusters
        tclusters[c_idx] = np.concatenate((np.array(tclusters[c_idx][1]).flatten(),np.array([np.concatenate((tclusters[c_idx][0][v_idx][0].flatten(),tclusters[c_idx][0][v_idx][1].flatten())) for v_idx in range(number_variables)]).flatten()))
    print('Tensor length:',len(tclusters[0])) #[len(i) for i in tclusters]) #...lengths should all be (num_dim^num_variables+denom_dim^num_variables)*num_variables + num_variables^2 <= (num+denom)*(all variables in clust) + exchange matrix

#Save data to a file
if save_data:
    with open('./A4D4_d'+str(depths)+'.txt','w') as file:
        file.write(str([[list(tclusters[clust].astype('int32')) for clust in range(sum(class_sizes[:c]),sum(class_sizes[:c+1]))] for c in range(len(class_sizes))]))
        

########### Additional Code ###########

In [ ]:
#Functionality to return number of seeds & quivers in EG when permutation equivalence applied
G = ClusterSeed(['A',4]) 
Q = ClusterSeed(['A',4]).quiver()
print('Size EG: ',len(G.mutation_class(up_to_equivalence=1)),'\nSize QEG:',len(Q.mutation_class(up_to_equivalence=1)))

In [ ]:
#Output the sparsity information for the tensor representations (run after generating data)
count=[]
for c in tclusters:
    if len(c) != len(tclusters[0]): print('inconsistent lengths!')
    count.append(0)
    for i in c:
        if i != 0: count[-1] += 1
print('Avg # of non-zero entries:',np.mean(count),', Avg proportion:',np.mean(count)/len(tclusters[0]))